In [9]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

train = pd.read_csv('./pretest/train_building37.csv').drop(columns=[ 'week', 'THI', 'CDH', 'max_power', 'min_power' ])
valid = pd.read_csv('./pretest/pretest_building37.csv')
train.shape



(1872, 19)

In [10]:
import torch

def sliding_window(tensor, window_size, step_size=1, batch_size=24):
    windows = []
    for i in range(0, tensor.size(0) - window_size + 1, step_size):
        windows.append(tensor[i:i+window_size])
    windows = torch.stack(windows)
    
    # 배치로 나누기
    num_batches = windows.size(0) // batch_size
    windows = windows[:num_batches * batch_size]
    windows_batched = windows.view(batch_size, -1, *windows.shape[1:])
    
    return windows_batched

# 예제 데이터
data_18 = torch.randn(1872, 18)
data_1 = torch.randn(1872, 1)

window_size = 42

batched_data_18 = sliding_window(data_18, window_size)
batched_data_1 = sliding_window(data_1, window_size)

print(batched_data_18.shape)  # [24, 42, 18]
print(batched_data_1.shape)   # [24, 42, 1]


torch.Size([24, 76, 42, 18])
torch.Size([24, 76, 42, 1])


In [8]:
# 이제 Tensor 형태의 데이터를 모델에 전달합니다.
x_train = train.drop(columns=['power_consumption'])
y_train = train['power_consumption']


#312
x_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values[:, None], dtype=torch.float32)
batch = 24
sequence_length = 48
feature_embed_dim = 18

# class Transformers(nn.Module):
#     def __init__(self, input_dim,d_model,nhead,encoder_layers):
#         super(Transformers,self).__init__()
        
#     def forward()

In [5]:

class TransformerModel(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, max_seq_length):
        super(TransformerModel, self).__init__()
        
        # Embedding Layer: 시계열 데이터를 d_model 차원으로 임베딩합니다.
        self.embedding = nn.Linear(input_dim, d_model)
        
        # Positional Encoding: 시퀀스의 위치 정보를 제공합니다.
        self.pos_encoder = nn.Embedding(max_seq_length, d_model)
        
        # Transformer Layer: Encoder와 Decoder로 구성됩니다.
        self.transformer = nn.Transformer(d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward)
        
        # Output Layer: 예측값을 출력합니다.
        self.out = nn.Linear(d_model, 168)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None, src_key_padding_mask=None, tgt_key_padding_mask=None):
        # src와 tgt는 각각 (S, N, E)와 (T, N, E)의 shape을 가집니다.
        # S는 source sequence length, T는 target sequence length, N은 batch size, E는 feature number입니다.
        
        # Embedding
        src = self.embedding(src) * torch.sqrt(torch.tensor(self.embedding.out_features, dtype=torch.float32))
        tgt = self.embedding(tgt) * torch.sqrt(torch.tensor(self.embedding.out_features, dtype=torch.float32))
        
        # Positional Encoding
        src_positions = (torch.arange(0, src.size(0)).unsqueeze(1).expand(src.size(0), src.size(1))).to(src.device)
        tgt_positions = (torch.arange(0, tgt.size(0)).unsqueeze(1).expand(tgt.size(0), tgt.size(1))).to(tgt.device)
        src = src + self.pos_encoder(src_positions)
        tgt = tgt + self.pos_encoder(tgt_positions)
        
        # Transformer
        output = self.transformer(src, tgt, src_mask, tgt_mask, None, None, src_key_padding_mask, tgt_key_padding_mask)
        
        # Output Layer
        return self.out(output)

# 예제
input_dim = 1872 # 예를 들어, 전력소비량만을 입력으로 사용한다면 1
d_model = 18
nhead = 6

num_encoder_layers = 6
num_decoder_layers = 6
dim_feedforward = 2048
max_seq_length = 2040
#모델선언
model = TransformerModel(input_dim, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, max_seq_length)





output = model(x_train_tensor, y_train_tensor)



IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)